<a href="https://colab.research.google.com/github/karoldem/comicfinder/blob/master/comicfinder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dear reader,

what you see is some unordered fragments of code to be stolen from.

It is for webscraping comics pages, performing OCR on images and saving results to a file.

In [1]:
!sudo apt install tesseract-ocr
!pip install pytesseract

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 39 not upgraded.
Need to get 4,795 kB of archives.
After this operation, 15.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-eng all 4.00~git24-0e00fe6-1.2 [1,588 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-osd all 4.00~git24-0e00fe6-1.2 [2,989 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr amd64 4.00~git2288-10f4998a-2 [218 kB]
Fetched 4,795 kB in 1s (5,254 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl

In [3]:
from requests import get
from bs4 import BeautifulSoup
from pytesseract import image_to_string
from json import load, dump
from io import BytesIO

try:
    from PIL import Image
except ImportError:
    import Image

savefile = True
if savefile:
    from google.colab import drive
    drive.mount('/content/gdrive')

class comicFinder:
    
    databaseSource = '/content/gdrive/My Drive/comictranscriotion.txt'

    def appendToJSONFile(self,newKey, newValue):

        with open(self.databaseSource, "r") as jsonFile:
            data = load(jsonFile)

        data[ newKey ] = newValue
        
        with open(self.databaseSource, "w") as jsonFile:
            dump(data, jsonFile)

    def extractText(self):
      
      lastComicURL = self.lastComicChecked()
      url = self.lastComicPosted()

      while not url == lastComicURL:
          
          comicElem = []

          for i in range(2):
            res = get(url)
            res.raise_for_status()
            soup = BeautifulSoup(res.text)
            comicElem = self.getComicElem(soup)
            if comicElem != []: break

          if not comicElem == []:
              try:
                    comicUrl = self.getComicUrl(comicElem)
                    res = get(comicUrl)
                    res.raise_for_status()
                    img = Image.open(BytesIO(res.content))

                    comicText = image_to_string(img)
                    self.appendToJSONFile(url,comicText.capitalize())

              except Exception as e:
                    print (e)

          try: 
            url = self.selectNextComic(soup)
          except:
              break



Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [7]:
class xkcd (comicFinder):

    lastComicCheckedFakeFile = 'http://xkcd.com/2596/'

    lastComicChecked = lambda self: self.lastComicCheckedFakeFile
    def updateLastComicChecked (self, url): self.lastComicCheckedFakeFile = url

    selectNextComic = lambda self, x :'http://xkcd.com' +  x.select('a[rel="prev"]')[0].get('href')
    selectPreviousComic = lambda self, x :'http://xkcd.com' +  x.select('a[rel="next"]')[0].get('href')
    getComicElem = lambda self, x : x.select('#comic img')
    getComicUrl = lambda self, x: 'http:' + x[0].get('src')

    def lastComicPosted(self):
        res = get('http://xkcd.com/')
        soup = BeautifulSoup(res.text)
        url = self.selectNextComic(soup)
        res = get(url)
        soup = BeautifulSoup(res.text)
        url = self.selectPreviousComic(soup)
        self.updateLastComicChecked(url)
        return url

In [9]:
x = xkcd()
x.extractText()
x.lastComicCheckedFakeFile

'http://xkcd.com/2597/'

# Archive

In [ ]:
import requests, os, bs4
from io import BytesIO
import pytesseract
import shutil
import os
import random
import json
import operator

try:
    from PIL import Image
except ImportError:
    import Image

savefile = False
if savefile:
    from google.colab import drive
    drive.mount('/content/gdrive')

def appendToJSONFile(newKey, newValue):

    with open('/content/gdrive/My Drive/comictranscriotion.txt', "r") as jsonFile:
        data = json.load(jsonFile)

    data[ newKey ] = newValue
    
    with open('/content/gdrive/My Drive/comictranscriotion.txt', "w") as jsonFile:
        json.dump(data, jsonFile)


class comicOCRer:


    def __init__(self, pageUrl, firstComicURL, lastComicURL, findThoseWordsInText):
        
        self.pageUrl = pageUrl
        self.firstComicURL = firstComicURL
        self.lastComicURL =  lastComicURL
        self.findThoseWordsInText = findThoseWordsInText
        
        if pageUrl == 'http://xkcd.com':
            self.selectNextComic = lambda x : 'http://xkcd.com' + x.select('a[rel="prev"]')[0].get('href')
            self.selectPreviousComic = lambda x : 'http://xkcd.com' + x.select('a[rel="next"]')[0].get('href')
            self.getComicElem = lambda x : x.select('#comic img')
            self.getComicUrl = lambda x: 'http:' + x[0].get('src')
            self.checkLastComic = self.checkLastComicXkcd

        if pageUrl == 'http://smbc-comics.com':
            self.selectNextComic = lambda x : x.select('a[rel="prev"]')[0].get('href')
            self.selectPreviousComic = lambda x : x.select('a[rel="next"]')[0].get('href')
            self.getComicElem = lambda x : x.select('#cc-comicbody img')
            self.getComicUrl = lambda x: x[0].get('src')
            self.checkLastComic = self.checkLastComicXkcd


    def run(self):
        return self.extractText(
                self.pageUrl, 
                self.firstComicURL, 
                self.lastComicURL, 
                self.findThoseWordsInText, 
                self.selectNextComic,
                self.getComicElem)

    turnOffOCR = False
    
    def checkLastComicXkcd(self):
        res = requests.get(self.pageUrl)
        res.raise_for_status()
        soup = bs4.BeautifulSoup(res.text)
        url = self.selectNextComic(soup)
        res = requests.get(url)
        res.raise_for_status()
        soup = bs4.BeautifulSoup(res.text)
        return self.selectPreviousComic(soup)

    def extractText(self, pageUrl, firstComicURL, lastComicURL, findThoseWordsInText, selectNextComic, getComicElem):

      wordCounter = {}
      transcription = {}
      url = firstComicURL

      while not url == lastComicURL:
          comicElem = []
          for i in range(2):
            res = requests.get(url)
            res.raise_for_status()
            soup = bs4.BeautifulSoup(res.text)
            comicElem = self.getComicElem(soup)
            if comicElem != []: break
            
          if comicElem == []:
              wordCounter[url] = 0
          else:
              try:
                    comicUrl = self.getComicUrl(comicElem)
                    res = requests.get(comicUrl)
                    res.raise_for_status()
                    img = Image.open(BytesIO(res.content))

                    if not self.turnOffOCR:
                      comicText = pytesseract.image_to_string(img)
                      if savefile:
                        appendToJSONFile(url,comicText.capitalize())
                      transcription[url] = comicText.capitalize()

                      for i in findThoseWordsInText:
                        if i in comicText:

                          try:
                            wordCounter[url] +=1
                          except:
                            wordCounter[url] = 1
                    else:
                      print (url)

              except Exception as e:
                    wordCounter[url] = 0
                    print(e)


          try: 
            url = selectNextComic(soup)
          except:
              break
      
      
      wordCounter = sorted(wordCounter.items(), key=operator.itemgetter(1), reverse=True)
      return wordCounter

In [ ]:
comicOCRer(
    pageUrl = 'http://smbc-comics.com',
    firstComicURL = 'http://smbc-comics.com',
    lastComicURL = 'https://www.smbc-comics.com/comic/i',
    findThoseWordsInText = ['direct', 'cultural', 'some', 'countries']
    ).run()

[]

In [ ]:
comicOCRer(
    pageUrl = 'http://xkcd.com',
    firstComicURL = 'https://xkcd.com',
    lastComicURL = 'hsdfghjkiuytgf',
    findThoseWordsInText = []
    ).run()